# General Information
This python code using flight data to obtain constrain, cost function and pairings.
<br>
Using Paper:  _**Robust Crew Pairing for Managing Extra Flights**_
<br>
Bases are; <br \>
    1 - Ankara <br \>
<br>
Constraints are;

|      Name      |        Description            |    Value     |
|----------------|-------------------------------|--------------|
| $t^{min}_{s}$  |  Minimum Sit Time             |  $60$  min   |
| $t^{max}_{s}$  |  Maximum Sit Time             |  $240$ min   |
| $t^{min}_{r}$  |  Minimum Rest Time            |  $600$ min   |
| $t^{max}_{r}$  |  Maximum Rest Time            |  $980$ min   |
| $N_{p}      $  |  Total Day in one Pairing     |  $4$ Day     |
| $N_{d}      $  |  Number of flight in one Duty |  $3$ Flight  |
| $T_{d}      $  |  Max Total (1- Duty) Time     |  $7$ Hour    |
| $T_{f}      $  |  Max Total (1- Duty) Time     |  $5$ Hour    | 

Earnings are;

# Python Data Prepering
## Import and From Libraries

In [ ]:
import pandas as pd           # Importing Pandas Library
import numpy as np            # Importing Numpy Library
from datetime import datetime # Importing Date Time to allow us doing math between two time
import scipy.io as sio        # Importing Sio to allow saving our mat files.

## Reading CSV Flight Data

In [ ]:
DataPath   = r'Data File/FlightDataSet38.csv'             # Definition of Data Path
FlightData = pd.read_csv(DataPath, sep=',', dtype=str)  # Flight Data reading from csv file
FlightData.head(5)                                      # Visualize first 5 row

## Chaning Data Type to Array and Obtaining Links

In [ ]:
FlightData = FlightData.values # Changing Data Type to Array

In [ ]:
# Saving Arrival and Departures
Arrival = FlightData[:,2]
Departure = FlightData[:,1]
Dep_Flight_Ank = np.array(np.where(Departure == 'ANT'))
Arr_Flight_Ank = np.array(np.where(Arrival =='ANT'))
foo = []
for i in range(Dep_Flight_Ank.shape[1]):
    for j in range(Arr_Flight_Ank.shape[1]):
        foo.append((Dep_Flight_Ank[0][i],Arr_Flight_Ank[0][j])) 

In [ ]:
# Obtaining Links with for loop
Flight_links = []
for i in range(len(FlightData)):
    for j in range(len(FlightData)):
        if FlightData[i][2] == FlightData[j][1]:
            Flight_links.append((FlightData[i,1:5], FlightData[j,1:5], 'Flight IDs :', i+1,j+1))

In [ ]:
Flight_links

## Using Sit Time Constraint to Eliminate Unnecessary Links
Minimum Sitting Time is 60 min. <br \>
Maximum Sitting Time is 240 min.  <br \>
According to these constraints we can eliminate links with using for loops

In [ ]:
# New Links with Constarints
Cons_Flight_links = []
Time_Format = '%H:%M'
for i in range(len(Flight_links)):
    # Looking Sit Time
    if (datetime.strptime(Flight_links[i][1][2], Time_Format) - datetime.strptime(Flight_links[i][0][3], Time_Format)).seconds/60 > 60 and (datetime.strptime(Flight_links[i][1][2], Time_Format) - datetime.strptime(Flight_links[i][0][3], Time_Format)).seconds/60 < 240:
        Cons_Flight_links.append(Flight_links[i])
    # Looking Rest Time
    elif (datetime.strptime(Flight_links[i][1][2], Time_Format) - datetime.strptime(Flight_links[i][0][3], Time_Format)).seconds/60 > 600 and (datetime.strptime(Flight_links[i][1][2], Time_Format) - datetime.strptime(Flight_links[i][0][3], Time_Format)).seconds/60 < 980:
        Cons_Flight_links.append(Flight_links[i])

In [ ]:
Cons_Flight_links

## Using Iterations to obtain Links with Different Initial Flights
##### This code is obtained from Ozgun's projects directly just changing variables name.

In [ ]:
# Allocation for iterations
iterations_start = np.zeros(len(Cons_Flight_links), dtype=np.uint32)
iterations_end = np.zeros(len(Cons_Flight_links), dtype=np.uint32)
#-----------------------------------------------------------------#  
# Iteration Process
for i in range(len(Cons_Flight_links)):
    iterations_start[i] = Cons_Flight_links[i][3]
    iterations_end[i] = Cons_Flight_links[i][4]
#-----------------------------------------------------------------#  
# Get rid of unnecessary initial flgihts
iterations_start_unique = np.unique(iterations_start)
unique, counts = np.unique(iterations_start, return_counts=True)
occurences =  dict(zip(unique, counts))
#-----------------------------------------------------------------#  
# Total with 16 bit unit
total = np.zeros(len(occurences)+1, dtype=np.uint16)
total[0] = 0
#-----------------------------------------------------------------#  
# Obtainin Arrival Location for One Departure
for i in range(len(occurences)):
    total[i+1] = total[i] + counts[i]
#-----------------------------------------------------------------#    
# Allocation for Multiple 1- Links    
MLinks_Flight = {}
for i in range(len(iterations_start_unique)):
    Un_Iter = iterations_start_unique[i]
    Val_Iter = iterations_end[total[i]:total[i+1]]
    Val_Iter = set(Val_Iter)
    MLinks_Flight[Un_Iter] = Val_Iter

In [ ]:
MLinks_Flight

## Obtaining Pairs with using "Depth Search"

In [ ]:
# Definition of Depth Search
def dfs_paths(graph, start, goal):
    stack = [(start, [start])]
    while stack:
        (vertex, path) = stack.pop()
        if vertex in MLinks_Flight:
            for next in graph[vertex] - set(path):
                if next == goal:
                    yield path + [next]
                else:
                    stack.append((next, path + [next]))
                    if len(stack) == 4:
                        exit()
        else:
            pass

In [ ]:
# Obtaining Pairs starting with ANT, ending with ANT
pairing = []
for i in range(len(foo)):
    pairing.append((dfs_paths(MLinks_Flight, foo[i][0], foo[i][1])))
    

In [ ]:
for i in range(len(pairing)):
    pairing[i] = list(pairing[i])

## Eliminate Unnecessary Pairings
1- Max number of flight is $3$. <br \>
2- Max number of duty in pairings $4$. <br \>
Thus, maximum number of flight in pairins must be $12$

In [ ]:
# First Elimination with maximum flight in pairs
Ele1_Pairs_Ist = []
for i in range(len(Pairs_IST)):
    if len(Pairs_IST[i]) < 12:
        Ele1_Pairs_Ist.append(Pairs_IST[i])

In [ ]:
# Second Elimination with duties
#'Minmum Number of duty is 2 So if we have single flight in duty so we eliminate these pairings '
#Ele2_Pairs_Ist = []
#for i in range(len(Ele1_Pairs_Ist)):
#    if (datetime.strptime(FlightData[Ele1_Pairs_Ist[0][1]-1][3], Time_Format) - (datetime.strptime(FlightData[Ele1_Pairs_Ist[0][0]-1][4], Time_Format))).seconds/60 < 540:
#        Ele2_Pairs_Ist.append(Ele1_Pairs_Ist[i])

## Cost Calculation
1- We need total flight time in a pairings <br>
2- For one pairins 1 hour in flight is $500$ TL. <br>
3- For one pairings if minutes between flight is less than 2 hour than its cost is $200$ TL. <br>

## Obtaining Matrix
Obtaining A, B, C Matrices with our pairings and flights. <br \>
A Matrix $\Rightarrow$ FlightData x Pairing List <br \>
B Matrix $\Rightarrow$ FlightData x 1 with ones  <br \>
C Matrix $\Rightarrow$ 1 x Pairing List with random numbers between range $5$ and $50$  <br \>

In [ ]:
# A Matrix
Amat = np.zeros((len(FlightData),len(Ele1_Pairs_Ist)), dtype=np.int16)
for i in range(len(Ele1_Pairs_Ist)):
    for j in range(len(Ele1_Pairs_Ist[i])):
        Save_Indices = Ele1_Pairs_Ist[i][j]-1
        Amat[Save_Indices][i] = 1
Amat = Amat[~(Amat==0).all(1)]
#-----------------------------------------------------------------#  
# B Matrix
Bmat = np.ones((Amat.shape[0],1), dtype=np.int16)
#-----------------------------------------------------------------#
# C Matrix
Cmat = np.zeros(Amat.shape[1], dtype=np.int16)
for i in range(len(Cmat)):
    Cmat[i] = np.random.uniform(5, 25)
#-----------------------------------------------------------------#
# Saving Matrices as Mat File
sio.savemat('Cmat.mat', {'Cmatrix': Cmat})
sio.savemat('Amat.mat', {'Amatrix': Amat})
sio.savemat('Bmat.mat', {'Bmatrix': Bmat})